In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
def read_test_data():
    Dfs = []
    train_df = []
    coords = pd.read_csv("data_phase_one/coords.csv", header=None)
    for name in ["mslp", "sst", "ssta", "t2m"]:
        df = pd.read_csv(f"./data_phase_one/test_input_{name}_phase_one.csv", header=None)
        print("Shape df", df.shape)
        Dfs.append(df)
        #Dfs.append(df.T.reset_index(drop=True))
    j = 0
    for i in range(0, df.shape[0], 12):
        feat = pd.concat([coords, Dfs[0].iloc[i:i+12], Dfs[1].iloc[i:i+12], Dfs[2].iloc[i:i+12], Dfs[3].iloc[i:i+12]], axis=0).reset_index(drop=True)
        feat = feat.T.reset_index(drop=True)
        feat.columns = [f"feat_{k}" for k in range(feat.shape[1])]
        if j < 3:
            print(feat.columns)
        train_df.append(feat)
        j += 1
    train_df = pd.concat(train_df, axis=0)
    return train_df

In [4]:
test_data = read_test_data()

Shape df (852, 5774)
Shape df (852, 5774)
Shape df (852, 5774)
Shape df (852, 5774)
Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12',
       'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18',
       'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24',
       'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30',
       'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36',
       'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42',
       'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47', 'feat_48',
       'feat_49'],
      dtype='object')
Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12',
       'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18',
       'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24

In [5]:
test_data.shape

(409954, 50)

In [6]:
import joblib

In [7]:
#lgb_model = joblib.load("./trained_models/lgb_model.joblib")
#catb_model = joblib.load("./trained_models/catb_model.joblib")

In [8]:
#y_pred1 = lgb_model.predict(test_data.values)
#y_pred2 = catb_model.predict(test_data.values)

In [9]:
#y_pred = 0.2*y_pred1 + 0.8*y_pred2 ==> -0.06
#y_pred = 0.55*y_pred1 + 0.45*y_pred2 ==> -0.03
#y_pred = 0.8*y_pred1 + 0.2*y_pred2 ==> -0.01
#y_pred = 0.95*y_pred1 + 0.05*y_pred2 ==> -0.0012
#y_pred = 0.99*y_pred1 - 0.01*y_pred2 ==> 0.0032

In [10]:
#y_pred = 0.99*y_pred1 - 0.01*y_pred2

In [11]:
#pred_df = pd.DataFrame(y_pred.reshape(71, 5774))

In [12]:
#pred_df

In [13]:
#pred_df.to_csv("./predictions.csv", index=False, header=False)

## K-Fold Predictions

In [14]:
import os
from sklearn.base import BaseEstimator, RegressorMixin

In [15]:
class VotingModel(BaseEstimator, RegressorMixin):
    def __init__(self, estimators, coeffs):
        super().__init__()
        self.estimators = estimators
        self.coeffs = coeffs
        
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        y_preds = [coeff*estimator.predict(X) for estimator, coeff in zip(self.estimators, self.coeffs)]
        return np.sum(y_preds, axis=0)

In [16]:
def load_boosting_models(path, num_folds=5):
    Models = []
    for fold in range(num_folds):
        for model_name in ["catb", "lgb"]:
            for trained_model_path in os.listdir(path):
                if trained_model_path.startswith("."): continue
                if model_name in trained_model_path and f'fold{fold}' in trained_model_path:
                    model = joblib.load(f'{path}/{trained_model_path}')
                    Models.append(model)
    return Models

In [26]:
boosting_models = load_boosting_models("./kfold_trained2/", 5)
coeffs = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

boosting_models = VotingModel(boosting_models, coeffs)

In [27]:
boosting_models

VotingModel(coeffs=[0.04, 0.16, 0.04, 0.16, 0.04, 0.16, 0.04, 0.16, 0.04, 0.16],
            estimators=[<catboost.core.CatBoostRegressor object at 0x7f43cb19ebd0>,
                        LGBMRegressor(device='gpu', gpu_device_id=1,
                                      learning_rate=0.01, max_depth=8,
                                      metric='rmse', n_estimators=2000,
                                      num_leaves=255, objective='regression',
                                      random_state=0, verbose=-1),
                        <catboost.core.CatBoostRegressor object at...
                                      learning_rate=0.01, max_depth=8,
                                      metric='rmse', n_estimators=2000,
                                      num_leaves=255, objective='regression',
                                      random_state=3, verbose=-1),
                        <catboost.core.CatBoostRegressor object at 0x7f43b7adc7d0>,
                        LGBMRegressor(device='gpu', gpu_device_id=1,
                                      learning_rate=0.01, max_depth=8,
                                      metric='rmse', n_estimators=2000,
                                      num_leaves=255, objective='regression',
                                      random_state=4, verbose=-1)])

In [19]:
#print(sum(boosting_models.coeffs))

In [28]:
y_pred = boosting_models.predict(test_data.values)
pred_df = pd.DataFrame(y_pred.reshape(71, 5774))

In [21]:
pred_df

,0,1,2,3,4,5,6,7,8,9,...,5764,5765,5766,5767,5768,5769,5770,5771,5772,5773
0,0.415247,0.609742,0.503264,0.261806,0.197564,0.195448,0.172960,0.203128,0.173025,0.169350,...,0.013491,-0.017725,0.148130,0.114392,0.047298,0.109559,0.283891,0.130067,0.545270,0.747813
1,-0.465346,-0.512959,-0.482990,-0.461780,-0.600697,-0.514868,-0.422448,-0.364278,-0.317668,-0.385797,...,-0.212585,-0.211821,0.061770,0.166970,0.244622,0.302895,0.343881,0.376082,0.495570,0.867060
2,-0.093231,-0.052662,-0.025566,-0.063567,-0.055240,-0.058757,-0.052953,-0.041010,-0.039486,-0.042737,...,0.031812,0.029771,0.231616,0.290782,0.334283,0.345708,0.420514,0.349898,0.430698,0.367511
3,-0.125673,-0.181400,-0.099775,-0.112650,-0.079238,-0.025680,-0.014986,-0.024038,-0.013635,-0.018228,...,-0.326608,-0.252101,-0.159040,-0.105152,-0.047870,0.030378,-0.071861,-0.103369,-0.000931,-0.054199
4,-0.539399,-0.333019,-0.336925,-0.454047,-0.443471,-0.348264,-0.271746,-0.257292,-0.392279,-0.407504,...,-0.447019,-0.482028,-0.278163,-0.130700,-0.012827,0.060881,-0.019398,0.121631,0.336582,0.046348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,0.517943,0.421148,0.369496,0.234380,0.101029,0.063169,0.079739,0.147141,0.117947,0.125366,...,-0.171049,-0.202178,0.096185,0.216509,0.254911,0.362725,0.379895,0.327691,0.532567,0.330308
67,0.164444,0.206723,0.227935,0.127268,0.089756,0.065262,0.079586,0.076502,0.076574,0.047625,...,-0.149556,-0.066936,0.159253,0.287279,0.325079,0.363594,0.467640,0.277732,0.444255,0.369314
68,-0.262072,-0.247581,-0.248435,-0.269852,-0.228721,-0.287209,-0.229858,-0.175299,-0.198542,-0.172039,...,0.446681,0.447990,0.570705,0.592454,0.493741,0.482713,0.749562,0.619133,0.694273,0.452371
69,0.037581,-0.021504,0.055414,0.042652,0.041550,0.042626,0.047514,0.040171,0.029384,0.003028,...,-0.115120,-0.047807,0.322617,0.357677,0.374283,0.316097,0.362342,0.148207,0.257601,-0.071785


In [30]:
pred_df.to_csv("./predictions.csv", index=False, header=False)

In [23]:
# KFold

# catb 500 estimators, lgbm 1000 estimators
# [0.05, 0.15, 0.05, 0.15, 0.05, 0.15, 0.05, 0.15, 0.05, 0.15] ==> 0.093

# catb and lgb 1000 estimators
# [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1] ==> 0.094